In [1]:
# Remove Tensorflow 2.0 Error Messages
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import datetime
import time

from pathlib import Path

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import *
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from tensorflow import keras
from tensorflow.keras import callbacks

import tensorflow.keras.backend as kb
from tensorflow.python.ops import *

import random

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("tensorflow").addHandler(logging.NullHandler(logging.ERROR))

import numpy as np
from IPython.display import display
from bqplot import (
    OrdinalScale, LinearScale, Bars, Lines, Axis, Figure, PanZoom, Toolbar
)

from random import shuffle

import ipywidgets as widgets

import pickle

lrelu = lambda x: tf.keras.layers.LeakyReLU(alpha=0.1)(x)

class Ein_Multiply(tf.keras.layers.Layer):
    def __init__(self, name=None, **kwargs):
        super(Ein_Multiply, self).__init__()

    def call(self, input):
        return tf.einsum('ij,ik->ik', input[0], input[1])

class Ein_Multiply_Add_None(tf.keras.layers.Layer):
    def __init__(self, name=None, **kwargs):
        super(Ein_Multiply_Add_None, self).__init__()

    def call(self, input):
        # Multiply the output embeddings with the GCN NAM
        # Then add another None dimension to satisfy the LSTM requirements
        o = tf.einsum('ij,ik->ik', input[0], input[1])[None, :, :]

        # Swap the None Dimensions
        return tf.einsum("ij...->ji...", o)
    

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
from os import listdir
from os.path import isfile, join

data_folder = Path(r"G:\Shared drives\Max Huffman - ECEN 403 404 URS Research 2020 2021\Datasets\ECEN403_Data\2013-01-01 - NASDAQ - Cleaned")

list_of_data_csv_files = [f for f in listdir(data_folder) if isfile(join(data_folder, f))]

# list_of_data_csv_files = [i[7:].replace('_1.csv','') for i in list_of_data_csv_files]

# Number of companies
N = len(list_of_data_csv_files)

# In this example we're cleaning up the csv files that are being read.
# We don't need the first column or the last row of the provided data set
df = pd.read_csv(data_folder / list_of_data_csv_files[0], header=None, usecols=[1, 2, 3, 4, 5])
df.columns = ['Close', '5D MA', '10D MA', '20D MA', '30D MA']

# Removethe bottom 5 rows and top 1 row to avoid -1234 empty spaces
df = df[1:-5]

# Number of sequences
S = (len(df.index))

# Number of features (columns)
D = len(df.columns)

# 3 Dimensional Tensor of the input data: (Companies, Sequences, Features)
XX_t = []

# Labels for the input data
# (Should be the closing price of each stock, but time shifted by 1 day)
YY_t = []

for stock in range(0, N):
    # Read its CSV values, skipping the index column and the last row which has -1234 as a value
    df = pd.read_csv(data_folder / list_of_data_csv_files[stock], header=None, usecols=[1, 2, 3, 4, 5], engine='python')
    df.columns = ['Close', '5D MA', '10D MA', '20D MA', '30D MA']
    
    # Removethe bottom 5 rows and top 1 row to avoid -1234 empty spaces
    df = df[1:-5]
    
    '''All stocks that contained missing/NaN values have already been removed from the dataset'''
    
    # Append the 2x2 tensor of each company's features through time to the main tensor
    XX_t.append(df[0:].values)
    YY_t.append(df['Close'].values)
    # print(f'Loading: {stock}, ', end='')


# Shift the Y values so that the future is time shifted by 1 day
for i in range(0, len(YY_t)):
    YY_t[i] = YY_t[i][1:]

# Remove the last X value since we don't have a predictor for it
for i in range(0, len(XX_t)):
    XX_t[i] = XX_t[i][0:-1]

In [6]:
# Convert the list of list of lists into a tensor object
XX_tf = tf.constant(XX_t, dtype='float32')
YY_tf = tf.constant(YY_t, dtype='float32')

In [7]:
def split_windows(total, percentages_list):
    # Get a sum of the initial total
    percentage_sum = sum(percentages_list)
    
    # Create a new list based on a percentage of the total
    new_splits = []
    for perc in percentages_list:
        new_splits.append(int(total * (perc / percentage_sum)))
    
    if sum(new_splits) != total:
        new_splits[0] = new_splits[0] + (total - sum(new_splits))
    
    return new_splits

# Split the data into 3 categories
x_train, x_val, x_test = tf.split(XX_tf, split_windows(XX_tf.shape[1], [60, 15, 25]), axis=1)
y_train, y_val, y_test = tf.split(YY_tf, split_windows(YY_tf.shape[1], [60, 15, 25]), axis=1)

print(x_train.shape, x_val.shape, x_test.shape)

TensorShape([881, 745, 5]) TensorShape([881, 185, 5]) TensorShape([881, 309, 5])


In [8]:
# Given a list, return the max index of that list
# If given rank, then it returns the rank-th highest index
def max_index(l, rank=0):
    items = []
    for idx, value in enumerate(l):
        items.append((idx, value))
    
    # Sort the ratios and their indexes from high to low
    items.sort(key= lambda x: x[1], reverse=True)
    
    return(items[rank][0])

# Given a list, return the min index of that list
def min_index(l):
    min_value = l[0]
    min_i = 0
    for i, v in enumerate(l):
        if min_value > v:
            min_value = v
            min_i = i
    return min_i

In [9]:
import json
# Create a list of the companies that are actually being trained over
# Only works with the EXPLICIT file format that we've currently provided
companies = [i[7:].replace('_1.csv','') for i in list_of_data_csv_files]

# Load in the JSON file as a DICT containing the industry relationships
with open(r'C:\Users\Maxwell\PycharmProjects\TAMU-ECEN-403-IFPTSND\Temporal_Relational_Stock_Ranking-master\data\RELATIONS\sector_industry\NASDAQ_industry_ticker.json') as read_file:
    given_industry_relations = json.load(read_file)

# Create a new dictionary of industry relationships containing only the companies we've trained over
new_industry_relations = {}
companies = set(companies)
for key, value in given_industry_relations.items():
    value = set(value)
    value.intersection_update(companies)
    value = list(value)
    
    # Only count relations that contain more than 1 stock
    if len(value) > 1:
        new_industry_relations[key] = value

# Create a json file containing the new relations to be used in other programs
filename = 'updated_relations.json'
with open(r'C:\Users\Maxwell\PycharmProjects\TAMU-ECEN-403-IFPTSND\Temporal_Relational_Stock_Ranking-master\data\RELATIONS\sector_industry\\' + filename, 'w') as write_file:
    json.dump(new_industry_relations, write_file)



# Ensure that companies is in the proper order
companies = list(companies)
companies.sort()

'''Given an item and a list, returns the index of that where that item is stored in a list'''
def return_idx(item, l):
    i = 0
    for thing in l:
        if item == thing:
            return i
        else:
            i += 1
    # If it's not in the list, then it's failed
    return -1

# Iterate through each company ticker and replace it with a tuple that contains its index and ticker
for key, value in new_industry_relations.items():
    new_value = []
    for v in value:
        new_value.append((return_idx(v, companies), v))
    new_industry_relations[key] = new_value

# Iterate through each industry relationship and create an N x N adjacency matrix
# Combine them all to create the final adjacency matrix in the same format as Paper #2
RR_t = []
for sector in new_industry_relations.keys():
    # Create an empty relationship matrix
    all_zeroes = tf.zeros([len(companies), len(companies)])
    relation_slice = all_zeroes.numpy()
    
    # Gather all the companies that exist in this sector
    siblings = new_industry_relations[sector]
    for i in siblings:
        for j in siblings:
            relation_slice[i[0], j[0]] = 1
            relation_slice[j[0], i[0]] = 1
    RR_t.append(relation_slice)

RR_tf = tf.constant(RR_t)

RR_tf = tf.transpose(RR_tf)

relation_encoding = RR_tf.numpy()
# relation_encoding.shape = [1024, 1024, 97]

rel_shape = [relation_encoding.shape[0], relation_encoding.shape[1]]

mask_flags = np.equal(np.zeros(rel_shape, dtype=int), np.sum(relation_encoding, axis=2))

# N x N
ajacent = np.where(mask_flags, np.zeros(rel_shape, dtype=float), np.ones(rel_shape, dtype=float))

# N x 1
degree = np.sum(ajacent, axis=0)
for i in range(len(degree)):
    degree[i] = 1.0 / degree[i]
np.sqrt(degree, degree)
deg_neg_half_power = np.diag(degree)

GCN_mat = np.dot(np.dot(deg_neg_half_power, ajacent), deg_neg_half_power)

# Some relation values in my degree matrix were nan, let's try replacing with 0
GCN_mat = np.nan_to_num(GCN_mat)
GCN_mat = tf.constant(GCN_mat)


ERROR! Session/line number was not unique in database. History logging moved to new session 690


<ipython-input-9-c77a06ad04fe>:84: RuntimeWarning: divide by zero encountered in double_scalars
  degree[i] = 1.0 / degree[i]


In [31]:
class ModelCompare():
    def __init__(self, test_set, increment):
        self.test_set = test_set.numpy()
        self.starting_investment = 2 * increment
        self.increment = increment
        
        self.num_companies = int(self.test_set.shape[0])
        self.num_days = int(self.test_set.shape[1])
        
        # Strategy_Name: [Daily portfolio value]
        self.strategy_results = {}
        
        # The amount of testing data we want the models to see before actually starting to spend money
        self.lag = 1
    
    def save_results(self, path):
        for key, value in self.strategy_results.items():
            pickle.dump(value, open (path + fr'\{key}.p', 'wb'))
    
    def simple_graph(self, sel_comp, show_all):
        
        # Create some colors for the graphed lines to cycle through
        colors_list = [hex(0x8A0808 - i *0xB0D0) for i in range(25)]
        colors_list = [str(i) for i in colors_list]
        colors_list = ['#' + i[2:] for i in colors_list]
        shuffle(colors_list)
    
        strategy_totals = self.strategy_results[sel_comp]

        if not show_all:
            x_data = list(range(0, len(strategy_totals)))
        if show_all:
            lengths = [len(i) for i in self.strategy_results.values()]
            x_data = list(range(0, max(lengths)))
        y_data = list(strategy_totals)

        x_scale = LinearScale()
        y_scale = LinearScale()

        ax_x = Axis(scale=x_scale, label='Trading Days', grid_lines='solid')
        ax_y = Axis(scale=y_scale, label='Portfolio Total', orientation='vertical', label_offset='50px', )

        if not show_all:
            line = [Lines(labels=[sel_comp], x=x_data, y=y_data, scales={'x': x_scale, 'y': y_scale}, display_legend=True)]
        else:
            values = [i for i in  self.strategy_results.values()]
            keys = [i for i in  self.strategy_results.keys()]
            line = [Lines(labels=[keys[i]], x=x_data, y=values[i], scales={'x': x_scale, 'y': y_scale}, colors=[colors_list[i]], display_legend=True) for i in range(len(values))]

        panzoom= PanZoom(scales={'x': [x_scale], 'y': [y_scale]})

        fig = Figure(marks=line, axes=[ax_x, ax_y], title='Total USD Earnings over Time', colors=['red'], legend_location='top-left', \
                     fit_margin={'top':100, 'bottom':100, 'left':100, 'right':100})

        toolbar = Toolbar(figure = fig)

        display(fig, toolbar)

    
    def graph(self):
        graphable_strats = [key for key in self.strategy_results]
        
        comp_drop = widgets.Dropdown(
            options = graphable_strats,
            description = 'Strategy:'
        )
        
        show_all = widgets.Checkbox(
            value=True,
            description = 'Show All',
            disabled=False,
        )
        
        widgets.interact(self.simple_graph, sel_comp = comp_drop, show_all = show_all)
        
    def buy_then_sell(self, company, day, amount):
        today_price = self.test_set[company, day, 0]
        tomorrow_price = self.test_set[company, day+1, 0]
        return amount * (tomorrow_price / today_price)
    
    def mse_sanity_check(self, path, tag='', exp_vis=False, name_override=''):
        
        # Name the strategy based on the path file
        strat_name = path.split('\\')[-1] 
        strat_name = strat_name + tag
        
        if name_override:
            strat_name = name_override
        
        print(f"\nLoading Model: '{strat_name}'")
        # Load in the given LSTM Model
        model = tf.keras.models.load_model(path, compile=False)
        
        pred_act_pairs = []
        for day in range(2, 200):
            print(day, end='')
            # The model should only be able to see up to yesterday
            seeable = self.test_set[:, 0:day, :]
            # Make a prediction
            pred = model.predict(tf.constant(seeable))
            todays_closes = [self.test_set[c, day, 0] for c in range(self.num_companies)]
            pred_act_pairs.append((pred, todays_closes))
        
        # Code for double checking that our model is making predictions
        '''
        company = 400
        actual_price = []
        pred_price = []
        for day in prices:
            pred = day[0]
            act = day[1]
            actual_price.append(act[company])
            pred_price.append(pred[company])

        plt.plot(actual_price)
        plt.plot(pred_price)
        
        return pred_act_pairs
        '''
    
    def strategy_follower(self):
        # Portfolio to start
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        for day in range(2, self.num_days - 2):
        
            if total < 0:
                break
            
            two_back_closes = [self.test_set[c, day-2, 0] for c in range(self.num_companies)]
            one_back_closes = [self.test_set[c, day-1, 0] for c in range(self.num_companies)]
            
            diff = [one_back_closes[i] - two_back_closes[i] for i in range(self.num_companies)]
            
            highest_earner = max_index(diff)
            
            print(f"Day: {day} Total: {round(total)} Company: {list_of_data_csv_files[highest_earner]}")
            
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            yesterday_earning = self.buy_then_sell(highest_earner, day, self.increment)
            
            total_by_day.append(total)
            
        self.strategy_results['Follower'] = total_by_day
        
    def strategy_best(self):
        # Portfolio to start
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        for day in range(2, self.num_days - 2):
        
            if total < 0:
                break
            
            today_closes = [self.test_set[c, day, 0] for c in range(self.num_companies)]
            tomorrow_closes = [self.test_set[c, day+1, 0] for c in range(self.num_companies)]
            
            diff = [tomorrow_closes[i] - today_closes[i] for i in range(self.num_companies)]
            
            highest_earner = max_index(diff)
            
            print(f"Day: {day} Total: {round(total)} Company: {list_of_data_csv_files[highest_earner]}")
            
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            yesterday_earning = self.buy_then_sell(highest_earner, day, self.increment)
            
            total_by_day.append(total)
            
        self.strategy_results['Follower'] = total_by_day
    
    def strategy_mse_lstm(self, path, tag='', exp_vis=False, name_override=''):
        
        # Name the strategy based on the path file
        strat_name = path.split('\\')[-1] 
        strat_name = strat_name + tag
        
        if name_override:
            strat_name = name_override
        
        print(f"\nLoading Model: '{strat_name}'")
        # Load in the given LSTM Model
        model = tf.keras.models.load_model(path, compile=False)
        
        # Portfolio to start
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        for day in range(self.lag, self.num_days - self.lag):
        
            if total < 0:
                break
                
            # The model should only be able to see up to yesterday
            seeable = self.test_set[:, 0:day, :]
            
            # Make a prediction
            pred = model.predict(tf.constant(seeable))
            
            todays_closes = [self.test_set[c, day, 0] for c in range(self.num_companies)]
            
            diff = [todays_closes[i] - pred[i] for i in range(self.num_companies)]
            
            highest_earner = max_index(diff)
            
            print(f"Day: {day} Total: {round(total)} Company: {list_of_data_csv_files[highest_earner]}")
            
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            yesterday_earning = self.buy_then_sell(highest_earner, day, self.increment)
            
            total_by_day.append(total)
            
        self.strategy_results[strat_name] = total_by_day
    
    def strategy_ratio_lstm(self, path, tag='', exp_vis=False, name_override=''):
        
        # Name the strategy based on the path file
        strat_name = path.split('\\')[-1] 
        strat_name = strat_name + tag
        
        if name_override:
            strat_name = name_override
        
        print(f"\nLoading Model: '{strat_name}'")
        # Load in the given LSTM Model
        model = tf.keras.models.load_model(path, compile=False, custom_objects = {'<lambda>': lrelu} )
        
        # Portfolio to start
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        for day in range(self.lag, self.num_days - self.lag):
        
            if total < 0:
                break
                
            # The model should only be able to see up to yesterday
            seeable = self.test_set[:, 0:day, :]
            
            # Make a prediction
            pred = model.predict(tf.constant(seeable))
            
            highest_earner = max_index(pred)
            
            print(f"Day: {day} Total: {round(total)} Company: {list_of_data_csv_files[highest_earner]}")
            
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            yesterday_earning = self.buy_then_sell(highest_earner, day, self.increment)
            
            total_by_day.append(total)
            
        self.strategy_results[strat_name] = total_by_day


    def strategy_ratio_lstm(self, path, name_override='', avoid_fall=False, average=1):
        
        # Name the strategy based on the path file
        strat_name = path.split('\\')[-1] 
        
        if name_override:
            strat_name = name_override
        
        print(f"\nLoading Model: '{strat_name}'")
        # Load in the given LSTM Model
        model = tf.keras.models.load_model(path, compile=False, custom_objects = {'<lambda>': lrelu} )
        
        # Portfolio to start
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        # Only used if avoid_fall strategy
        losing_streak = -1
        
        for day in range(self.lag, self.num_days - self.lag):
            
            if avoid_fall:
                if yesterday_earning < self.increment:
                    losing_streak += 1
            
            if total < 0:
                break
                
            # The model should only be able to see up to yesterday
            seeable = self.test_set[:, 0:day, :]
            
            # Make a prediction
            pred = model.predict(tf.constant(seeable))
            
            if avoid_fall:
                if yesterday_earning > self.increment:
                    c_choices = [max_index(pred, i) for i in range(average)]
                    losing_streak = 0
                else:
                    c_choices = [max_index(pred, i+losing_streak) for i in range(average)]
                    
            else:
                c_choices = [max_index(pred, i) for i in range(average)]
            
            print(f"Day: {day} Total: {round(total)} Companies: {[list_of_data_csv_files[c] for c in c_choices]}")
            
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            # Calculate the amount of money that will be earned tomorrow when sold
            sum = 0
            for c in c_choices:
                sum = sum + self.buy_then_sell(c, day, self.increment / len(c_choices))
            
            yesterday_earning = sum
            
            total_by_day.append(total)
            
        self.strategy_results[strat_name] = total_by_day
        
    
    def strategy_ratio_gcn(self, path, tag='', exp_vis=False, name_override='', avoid_fall=True):
        
        # Name the strategy based on the path file
        strat_name = path.split('\\')[-1] 
        strat_name = strat_name + tag
        
        if name_override:
            strat_name = name_override
        
        print(f"\nLoading Model: '{strat_name}'")
        # Load in the given LSTM Model
        model = tf.keras.models.load_model(path, compile=False, custom_objects = {'Ein_Multiply': Ein_Multiply, '<lambda>': lrelu, 'Ein_Multiply_Add_None': Ein_Multiply_Add_None})
        
        # Portfolio to start
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        losing_streak = -1
        
        for day in range(self.lag, self.num_days - self.lag):
            
            if yesterday_earning < self.increment:
                losing_streak += 1
            
            if total < 0:
                break
                
            # The model should only be able to see up to yesterday
            seeable = self.test_set[:, 0:day, :]
            
            # Make a prediction
            pred = model.predict([tf.constant(seeable), GCN_mat])
            
            if avoid_fall:
                if yesterday_earning > self.increment:
                    highest_earner = max_index(pred, 0)
                    losing_streak = 0
                else:
                    highest_earner = max_index(pred, losing_streak)
            else:
                highest_earner = max_index(pred, 0)
            
            print(f"Day: {day} Total: {round(total)} Companies: {[list_of_data_csv_files[c] for c in c_choices]}")
            
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            # Calculate the amount of money that will be earned tomorrow when sold
            sum = 0
            for c in c_choices:
                sum = sum + self.buy_then_sell(c, day, self.increment / len(c_choices))
            
            yesterday_earning = sum
            
            total_by_day.append(total)
            
        self.strategy_results[strat_name] = total_by_day
    
    def strategy_ratio_gcn(self, path, name_override='', avoid_fall=False, average=1, expVis=False):
        
        # Name the strategy based on the path file
        strat_name = path.split('\\')[-1] 
        
        if name_override:
            strat_name = name_override
        
        print(f"\nLoading Model: '{strat_name}'")
        # Load in the given LSTM Model
        # This line is dependent on knowing what custom layers or functions were used
        model = tf.keras.models.load_model(path, compile=False, custom_objects = {'Ein_Multiply': Ein_Multiply, '<lambda>': lrelu, 'Ein_Multiply_Add_None': Ein_Multiply_Add_None})
        
        # Portfolio to start
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        # Used in the avoid_fall scenario
        losing_streak = -1
        
        for day in range(self.lag, self.num_days - self.lag):
            
            if avoid_fall:
                if yesterday_earning < self.increment:
                    losing_streak += 1
            
            if total < 0:
                break
                
            # The model should only be able to see up to yesterday
            seeable = self.test_set[:, 0:day, :]
            
            if expVis:
                seeable = tf.concat([x_val, seeable], axis=1)
            
            # Make a prediction
            pred = model.predict([tf.constant(seeable), GCN_mat])
            
            if avoid_fall:
                if yesterday_earning > self.increment:
                    c_choices = [max_index(pred, i) for i in range(average)]
                    losing_streak = 0
                else:
                    c_choices = [max_index(pred, i+losing_streak) for i in range(average)]
                    
            else:
                c_choices = [max_index(pred, i) for i in range(average)]

            
            print(f"Day: {day} Total: {round(total)} Companies: {[list_of_data_csv_files[c] for c in c_choices]}")
            
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            # Calculate the amount of money that will be earned tomorrow when sold
            sum = 0
            for c in c_choices:
                sum = sum + self.buy_then_sell(c, day, self.increment / len(c_choices))
            
            yesterday_earning = sum
            
            total_by_day.append(total)
            
        self.strategy_results[strat_name] = total_by_day
    
    def strategy_average(self):
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        for day in range(self.lag, self.num_days - self.lag):
            
            # Choose 10 stocks at random
            c_choices = [random.randint(0, self.num_companies-1) for i in range(200)]
            
            # Gain the money from yesterday's purchase
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            # Calculate the amount of money that will be earned tomorrow when sold
            sum = 0
            for c in c_choices:
                sum = sum + self.buy_then_sell(c, day, self.increment / len(c_choices))
            
            yesterday_earning = sum
            
            total_by_day.append(total)
        
        self.strategy_results['Average'] = total_by_day 
    
    def strategy_random(self):
        total = self.starting_investment
        yesterday_earning = 0
        total_by_day = []
        
        for day in range(self.lag, self.num_days - self.lag):
            # Choose X stocks at random
            
            chosen1 = random.randint(0, self.num_companies-1)
            
            # Gain the money from yesterday's purchase
            total += yesterday_earning
            
            # Lose the money from the total that is spent today
            total -= self.increment
            
            # Calculate the amount of money that will be earned tomorrow when sold
            yesterday_earning = self.buy_then_sell(chosen1, day, self.increment)
            
            total_by_day.append(total)
        
        self.strategy_results['Random'] = total_by_day

In [32]:
A = ModelCompare(x_test, 50000)

In [38]:
path = r'G:\Shared drives\Max Huffman - ECEN 403 404 URS Research 2020 2021\Datasets\November_Models\\'

# A.strategy_random()
# A.strategy_average()
# A.strategy_average()
# A.strategy_best()
# A.strategy_follower()
# A.strategy_mse_lstm(path + 'easy)
# A.strategy_ratio_lstm(path + 'god', name_override='MSE-Ratio-104-Epoch;1e-5-LR-LeakyRelu-RankLoss')

# A.strategy_ratio_gcn_average(path + 'test', name_override='GCN-RankLoss-Ratio-33-Epoch-1NewLayer;3e-5-LR-Average2', avoid_fall=False, average=2)
# A.strategy_ratio_lstm(path + 'god', name_override='MSE-Ratio-104-Epoch;1e-5-LR-LeakyRelu-RankLoss')

# A.strategy_ratio_gcn(path + 'test', avoid_fall = False, name_override='GCN_RankLoss_Ratio_33epoch_1TLSTM1FLSTM_VarLR-expVis', average=1, expVis=True)
# A.strategy_ratio_gcn(path + 'test', avoid_fall = False, name_override='GCN_RankLoss_Ratio_33epoch_1TLSTM1FLSTM_VarLR-Average-2-expVis', average=2, expVis=True)
A.strategy_ratio_gcn(path + 'test', avoid_fall = True, name_override='GCN_RankLoss_Ratio_33epoch_1NewDense_VarLR-AvoidFall-expVis', average=1, expVis=True)


Loading Model: 'GCN_RankLoss_Ratio_33epoch_1NewDense_VarLR-AvoidFall-expVis'
Day: 1 Total: 100000 Companies: ['NASDAQ_SHEN_1.csv']
Day: 2 Total: 50000 Companies: ['NASDAQ_SHEN_1.csv']
Day: 3 Total: 50281.0 Companies: ['NASDAQ_SHEN_1.csv']
Day: 4 Total: 50537.0 Companies: ['NASDAQ_SHEN_1.csv']
Day: 5 Total: 50343.0 Companies: ['NASDAQ_HAIN_1.csv']
Day: 6 Total: 50412.0 Companies: ['NASDAQ_ZEUS_1.csv']
Day: 7 Total: 50273.0 Companies: ['NASDAQ_HAIN_1.csv']
Day: 8 Total: 52063.0 Companies: ['NASDAQ_ZEUS_1.csv']
Day: 9 Total: 51921.0 Companies: ['NASDAQ_HAIN_1.csv']
Day: 10 Total: 52751.0 Companies: ['NASDAQ_HAIN_1.csv']
Day: 11 Total: 52722.0 Companies: ['NASDAQ_HAIN_1.csv']
Day: 12 Total: 52631.0 Companies: ['NASDAQ_TSCO_1.csv']
Day: 13 Total: 52622.0 Companies: ['NASDAQ_LULU_1.csv']
Day: 14 Total: 52475.0 Companies: ['NASDAQ_SPWR_1.csv']
Day: 15 Total: 51881.0 Companies: ['NASDAQ_MDGL_1.csv']
Day: 16 Total: 51386.0 Companies: ['NASDAQ_GIII_1.csv']
Day: 17 Total: 53754.0 Companies: ['NA

Day: 146 Total: 48476.0 Companies: ['NASDAQ_FTR_1.csv']
Day: 147 Total: 48044.0 Companies: ['NASDAQ_LAWS_1.csv']
Day: 148 Total: 47269.0 Companies: ['NASDAQ_SPWR_1.csv']
Day: 149 Total: 47040.0 Companies: ['NASDAQ_MDGL_1.csv']
Day: 150 Total: 46713.0 Companies: ['NASDAQ_VIIX_1.csv']
Day: 151 Total: 46485.0 Companies: ['NASDAQ_SPWR_1.csv']
Day: 152 Total: 46073.0 Companies: ['NASDAQ_LAWS_1.csv']
Day: 153 Total: 46296.0 Companies: ['NASDAQ_LAWS_1.csv']
Day: 154 Total: 47695.0 Companies: ['NASDAQ_LAWS_1.csv']
Day: 155 Total: 49055.0 Companies: ['NASDAQ_LAWS_1.csv']
Day: 156 Total: 50219.0 Companies: ['NASDAQ_LAWS_1.csv']
Day: 157 Total: 51000.0 Companies: ['NASDAQ_FNSR_1.csv']
Day: 158 Total: 51549.0 Companies: ['NASDAQ_FNSR_1.csv']
Day: 159 Total: 51724.0 Companies: ['NASDAQ_MDGL_1.csv']
Day: 160 Total: 51646.0 Companies: ['NASDAQ_VIIX_1.csv']
Day: 161 Total: 51047.0 Companies: ['NASDAQ_HQCL_1.csv']
Day: 162 Total: 50633.0 Companies: ['NASDAQ_FTR_1.csv']
Day: 163 Total: 50156.0 Companies

Day: 291 Total: 77532.0 Companies: ['NASDAQ_FIZZ_1.csv']
Day: 292 Total: 76778.0 Companies: ['NASDAQ_TTS_1.csv']
Day: 293 Total: 77032.0 Companies: ['NASDAQ_TTS_1.csv']
Day: 294 Total: 76914.0 Companies: ['NASDAQ_CELG_1.csv']
Day: 295 Total: 76973.0 Companies: ['NASDAQ_ACHC_1.csv']
Day: 296 Total: 76926.0 Companies: ['NASDAQ_EFII_1.csv']
Day: 297 Total: 76384.0 Companies: ['NASDAQ_TTS_1.csv']
Day: 298 Total: 75010.0 Companies: ['NASDAQ_MACK_1.csv']
Day: 299 Total: 74891.0 Companies: ['NASDAQ_ESND_1.csv']
Day: 300 Total: 74328.0 Companies: ['NASDAQ_CELG_1.csv']
Day: 301 Total: 74493.0 Companies: ['NASDAQ_CELG_1.csv']
Day: 302 Total: 75046.0 Companies: ['NASDAQ_EFII_1.csv']
Day: 303 Total: 75514.0 Companies: ['NASDAQ_EFII_1.csv']
Day: 304 Total: 75866.0 Companies: ['NASDAQ_EFII_1.csv']
Day: 305 Total: 76036.0 Companies: ['NASDAQ_EFII_1.csv']
Day: 306 Total: 76570.0 Companies: ['NASDAQ_EFII_1.csv']
Day: 307 Total: 77164.0 Companies: ['NASDAQ_EFII_1.csv']


In [39]:
A.graph()
A.save_results(r'G:\Shared drives\Max Huffman - ECEN 403 404 URS Research 2020 2021\Datasets\Trading_Results')
# test = pickle.load(open(r'G:\Shared drives\Max Huffman - ECEN 403 404 URS Research 2020 2021\Datasets\Trading_Results\Average.p', 'rb'))

interactive(children=(Dropdown(description='Strategy:', options=('GCN_RankLoss_Ratio_33epoch_1TLSTM1FLSTM_VarL…

In [20]:
x = tf.concat([x_val, x_test], axis=1)

In [22]:
print(x_val.shape, x_test.shape)

TensorShape([881, 185, 5]) TensorShape([881, 309, 5])
